In [4]:
# ONE-CELL BASELINE (conservative per-ID) for your schema:
# receivals:  rm_id, date_arrival, net_weight
# mapping:    ID, rm_id, forecast_start_date, forecast_end_date
# sample:     ID, target

import pandas as pd
from pathlib import Path

# -------- Paths --------
ROOT = Path(__file__).resolve().parents[1] if "__file__" in locals() else Path.cwd().parent
DATA = ROOT / "data" / "kernel"
OUT  = ROOT / "output"

# -------- Robust date parsing --------
def to_dt_robust(series):
    """Parse to datetime (handles strings, yyyymmdd, mixed tz). Returns tz-naive datetime64[ns]."""
    s = series.astype(str).str.strip()
    # parse with utc=True to avoid mixed-tz issues, then drop tz to get naive
    out = pd.to_datetime(s, errors="coerce", utc=True)
    out = out.dt.tz_localize(None)
    # Fallback: if still many NaT and looks like yyyymmdd, try that format
    if out.isna().mean() > 0.2 and s.str.fullmatch(r"\d{8}").any():
        out = pd.to_datetime(s, format="%Y%m%d", errors="coerce", utc=True).dt.tz_localize(None)
    return out

# -------- Load --------
rec      = pd.read_csv(DATA / "receivals.csv")
po       = pd.read_csv(DATA / "purchase_orders.csv")              # not used in baseline
mapping  = pd.read_csv(DATA / "prediction_mapping.csv")
sample   = pd.read_csv(DATA / "sample_submission.csv")

# -------- Parse dates (your actual columns) --------
rec["date_arrival"]              = to_dt_robust(rec["date_arrival"])
mapping["forecast_start_date"]   = to_dt_robust(mapping["forecast_start_date"])
mapping["forecast_end_date"]     = to_dt_robust(mapping["forecast_end_date"])
po["delivery_date"]              = to_dt_robust(po["delivery_date"])  # kept for later use

# drop unparseable receivals (rare)
rec = rec.dropna(subset=["date_arrival"]).copy()

# -------- Daily totals & cumulative up to cutoff --------
rec["date"] = rec["date_arrival"].dt.date
daily = (rec.groupby(["rm_id", "date"], as_index=False)["net_weight"].sum()
           .sort_values(["rm_id","date"]))
daily["cum_qty"] = daily.groupby("rm_id")["net_weight"].cumsum()

# For each ID, predict the last known cumulative BEFORE its window starts
mapping["cutoff_date"] = (mapping["forecast_start_date"] - pd.Timedelta(days=1)).dt.date

# join each ID to its rm_id history and keep rows <= cutoff
joined = (mapping.merge(daily, on="rm_id", how="left")
                 .query("date <= cutoff_date"))

# last cumulative per ID
last_per_id = (joined.sort_values(["ID","date"])
                      .groupby("ID", as_index=False)["cum_qty"].last())

# IDs with no history → 0 (conservative)
last_per_id["cum_qty"] = last_per_id["cum_qty"].fillna(0)

# -------- Build submission --------
sub = (sample[["ID"]]
       .merge(last_per_id.rename(columns={"cum_qty":"target"}), on="ID", how="left")
       .fillna({"target": 0}))

OUT.mkdir(parents=True, exist_ok=True)
sub.to_csv(OUT / "submission_baseline_A.csv", index=False)

print("Wrote:", OUT / "submission_baseline_A.csv")
print("Rows:", len(sub))
display(sub.head())


Wrote: c:\Users\magnu\Datateknologi (fag)\TDT4173 Moderne maskinlæring i praksis - Prosjekt ML\append_consulting_project\append_consulting_project\output\submission_baseline_A.csv
Rows: 30450


,ID,target
0,1,25616003.0
1,2,25616003.0
2,3,25616003.0
3,4,25616003.0
4,5,25616003.0
